MOABB to CSV / RAW to CSV

This code convert the data sets in RAW format to CSV format.

It has been specifically conceived for BCI data.

This script is for Cho2017



In [ ]:
import numpy as np
import pandas as pd
import moabb.datasets
import os

In [ ]:
#Load Database
m_dataset = moabb.datasets.Cho2017()
subjects = m_dataset.subject_list
m_data = m_dataset.get_data(subjects = [subjects[0]])

In [ ]:
#See all canal names (EEG, misc, stim...)
raw = m_data[1]['0']['0']
# raws
print("Canal list :", raw.ch_names)

In [ ]:
#Know what index is stim channel (we will need it later for the CSV to NY conversion)
stim_channel_name = 'Stim'
stim_idx = raw.ch_names.index(stim_channel_name)
print(f"Canal index {stim_channel_name} is : {stim_idx}")


In [ ]:
#count stim data unique values (1 non target, 2 = target with a ratio needed of 5 to 1)
stim_data = raw.get_data(picks=stim_idx)
print(stim_data.shape)
unique_vals, counts = np.unique(stim_data, return_counts=True)

for val, count in zip(unique_vals, counts):
    print(f"Value : {val}, Occurences count : {count}")

In [ ]:
# Transpose to invert columns/lines
raw.drop_channels(['EMG1', 'EMG2', 'EMG3', 'EMG4'])
data = raw.get_data()
dataT = data.T
print(dataT.shape)

In [ ]:
# creating timestamps and header
n_times, n_channels = dataT.shape
timestamps = np.arange(n_times, dtype=int)
data_with_timestamp = np.column_stack((timestamps, dataT))
header = [""] + [str(i) for i in range(n_channels)]

# Removing decimals from timestamps
df = pd.DataFrame(data_with_timestamp, columns=header)
df.iloc[:, 0] = df.iloc[:, 0].astype(int)

In [ ]:
# Test to check csv file
output_dir = 'C:/Users/doumif/work/Prog'
filename = "data.csv"
filepath = os.path.join(output_dir, filename)

df.to_csv(filepath, index=False)

In [ ]:
m_dataset = moabb.datasets.Cho2017()
subject_list = list(m_data.keys())

In [ ]:
m_data = m_dataset.get_data(subjects=subject_list)

In [ ]:
for subject in subject_list:
    session_keys = sorted(m_data[subject].keys())
    for idx, session in enumerate(session_keys, start=1):
        raw_session = m_data[subject][session]['0']
        raw_session.drop_channels(['EMG1', 'EMG2', 'EMG3', 'EMG4'])
        data = raw_session.get_data()

        # Transposer pour obtenir dataT de forme (total_timesamples, n_channels)
        dataT = data.T
        n_times, n_channels = dataT.shape

        # Création de la colonne de timestamps
        timestamps = np.arange(n_times, dtype=int)
        datacsv = np.column_stack((timestamps, dataT))
        header = [""] + [str(i) for i in range(n_channels)]
        df = pd.DataFrame(datacsv, columns=header)
        df[""] = df[""].astype(int)

        stim_col = str(n_channels - 1)  # La dernière colonne contient les stimulations

        # Nommer le fichier
        subject_str = f"{int(subject):02d}"
        session_str = f"{idx:02d}"
        filename = f"subject_{subject_str}_session_{session_str}.csv"
        output_dir = 'C:/Users/doumif/work/Prog/Cho2017'
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False)

        # Affichage des informations avec les valeurs swappées
        events = df[stim_col].values
        n_lh = len(events[events == 1])  # Maintenant 1 est target
        n_rh = len(events[events == 2])  # Maintenant 2 est non-target
        print(f"\nFichier sauvegardé : {filename}")
        print(f"Nombre de Left hand (1): {n_lh}")
        print(f"Nombre de Right hand (2): {n_rh}")
